In [1]:
import io
import os
import panel as pn
import numpy as np
import pandas as pd
import param

pn.extension('vega', 'ace', 'jsoneditor', 'ipywidgets', sizing_mode='stretch_width')

In [2]:
from pathlib import Path

In [43]:
import sys
sys.path.append("..")
from incf.preprocess import preprocess as prep

def check_file(fname=None, value=None):
    s_id = prep.create_uuid()
    
    # sub-AA_desc-50healthy_weights.tsv
    if fname == 'weights.txt':
        val = create_layout({'name': s_id, 
                       'desc': 'default',
                       'fname': fname
                      })
        create_output_folder(sub=f'sub_{s_id}')
        
        fname = f'sub-{s_id}_desc-default_{os.path.basename(fname)}.tsv'
        string_io = io.StringIO(value.decode("utf8"))
        pd.read_csv(string_io, sep='\t').to_csv(os.path.join('../output', f'sub_{s_id}', 'net', fname), sep='\t',
                                                header=None, index=None)
        return val

        
def create_output_folder(path='../output', sub='sub_00'):
    # verify folders exist
    check_folders(path)
    
    # patient specific folders 
    sub = os.path.join(path, sub)
    net = os.path.join(sub, 'net')
    
    if not os.path.exists(sub):
        print(f'Creating folder `{sub}`...')
        os.mkdir(sub)
        
        print(f'Creating folder `{net}`...')
        os.mkdir(net)
        
    else:
        # TODO: add create new id creation 
        pass


def check_folders(path):
    eq    = os.path.join(path, 'eq')
    code  = os.path.join(path, 'code')
    coor  = os.path.join(path, 'coord')
    param = os.path.join(path, 'param')
    
    for p in [path, eq, code, coor, param]:
        if not os.path.exists(p):
            print(f'Creating folder `{os.path.basename(p)}`...')
            os.mkdir(p)
    
    read  = os.path.join(path, 'README.txt')
    part  = os.path.join(path, 'participants.tsv')
    desc  = os.path.join(path, 'dataset_description.json')
    chgs  = os.path.join(path, 'CHANGES.txt')
    
    for p in [read, part, desc, chgs]:
        if not os.path.exists(p):
            print(f'Creating file `{os.path.basename(p)}`...')
            Path(p).touch()
    

# create_output_folder()

In [51]:
def create_layout(subs=None):
    return f"""
    output/ <br>
        &emsp;|___ code <br>
        &emsp;|___ coord <br>
        &emsp;|___ eq <br>
        &emsp;|___ param <br>
        &emsp;{create_sub(subs)}
        &emsp;|___ README <br>
        &emsp;|___ CHANGES <br>
        &emsp;|___ dataset_description.json <br>
        &emsp;|___ participants.tsv
    """

def create_sub(sub):
    if sub is not None:
        if sub['fname'] == 'weights.txt':
            return f"""
                |___ sub-{sub['name']} <br>
                     &emsp;&emsp;&emsp;|__ net <br>
                         &emsp;&emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_weights.tsv <br>
                         &emsp;&emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_weights.json <br>
                     &emsp;&emsp;&emsp;|__ spatial <br>
                     &emsp;&emsp;&emsp;|__ ts  <br>
            """


        elif sub['fname'] == 'distances.txt':
            return f"""
                &emsp;|___ sub-{sub['name']} <br>
                     &emsp;&emsp;|__ net <br>
                         &emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_distances.tsv <br>
                         &emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_distances.json <br>
                     &emsp;&emsp;|__ spatial <br>
                     &emsp;&emsp;|__ ts  <br>
            """
    return ''
#         return f"""
#             |___ sub-{sub['name']}
#                  |__ net
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_distances.tsv
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_distances.json
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_weights.tsv
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_weights.json
#                  |__ spatial
#                  |__ ts 
#         """



In [52]:
class UploadFile(param.Parameterized):
#     data = param.DataFrame()   # maybe show tsv file as well
    
    file_input = param.Parameter()
    
    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.FileInput(), **params)
        self.static_text = pn.widgets.StaticText()
        
        
    @pn.depends("file_input.value", watch=True)
    def _parse_file_input(self):
        value = self.file_input.value
        fname = self.file_input.filename
        
        if value: 
            self.static_text.value = check_file(fname, value)
        else:
            print("error")
        
    def view(self):
        return pn.Column(
            "## Upload data",
            self.file_input,
            self.static_text
        )

upload = UploadFile()
upload_app_view = upload.view()
pn.serve(upload_app_view)

Launching server at http://localhost:60938


Creating folder `../output\sub_0EP0W`...
Creating folder `../output\sub_0EP0W\net`...


In [41]:
pn.Column(
    '### Generate Files',
    pn.widgets.StaticText(value=f'{create_layout()}')
)

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] StaticText(sizing_mode='stretch_width', value='\n    output/ <...)

In [130]:
# import sys
# sys.path.append("..")
# from incf.preprocess import preprocess as prep

# def check_file(fname=None, value=None):
#     s_id = prep.create_uuid()
    
#     # sub-AA_desc-50healthy_weights.tsv
#     if fname == 'weights.txt':
#         create_output_folder(sub=f'sub_{s_id}')
        
#         fname = f'sub-{s_id}_desc-default_{os.path.basename(fname)}.tsv'
#         string_io = io.StringIO(value.decode("utf8"))
#         pd.read_csv(string_io, sep='\t').to_csv(os.path.join('../output', f'sub_{s_id}', 'net', fname), sep='\t',
#                                                 header=None, index=None)
        

In [131]:
# def create_output_folder(path='../output', sub='sub_00'):
#     # verify folders exist
#     check_folders(path)
    
#     # patient specific folders 
#     sub = os.path.join(path, sub)
#     net = os.path.join(sub, 'net')
    
#     if not os.path.exists(sub):
#         print(f'Creating folder `{sub}`...')
#         os.mkdir(sub)
        
#         print(f'Creating folder `{net}`...')
#         os.mkdir(net)
        
#     else:
#         # TODO: add create new id creation 
#         pass


# def check_folders(path):
#     eq    = os.path.join(path, 'eq')
#     code  = os.path.join(path, 'code')
#     coor  = os.path.join(path, 'coord')
#     param = os.path.join(path, 'param')
    
#     for p in [path, eq, code, coor, param]:
#         if not os.path.exists(p):
#             print(f'Creating folder `{os.path.basename(p)}`...')
#             os.mkdir(p)
    
#     read  = os.path.join(path, 'README.txt')
#     part  = os.path.join(path, 'participants.tsv')
#     desc  = os.path.join(path, 'dataset_description.json')
#     chgs  = os.path.join(path, 'CHANGES.txt')
    
#     for p in [read, part, desc, chgs]:
#         if not os.path.exists(p):
#             print(f'Creating file `{os.path.basename(p)}`...')
#             Path(p).touch()
    

# # create_output_folder()

In [132]:
# gen_btn = pn.widgets.Button(name='Generate Files')
# gen_btn.on_click(create_output_folder)

In [133]:
# gen_btn

In [76]:
path = '../data/txt_files/weights.txt'


In [55]:
def create_layout(subs=None):
    print("""
    output/
        |___ code
        |___ coord
        |___ eq
        |___ param
    """, end='')
    
#     if 


def create_sub():
    return f"""
        |___ sub-name
             |__ net
                 |__ sub-name_desc-desc_distances.tsv
                 |__ sub-name_desc-desc_distances.json
                 |__ sub-name_desc-desc_weights.tsv
                 |__ sub-name_desc-desc_weights.json
             |__ spatial
             |__ ts 
    
    """



In [58]:
def preprocess_file(file):
    if file.filename == 'weights.txt':
        print('hehe')
        print(file)

In [90]:
@pn.depends(file_upload, watch=True)
def set_subs(file_upload):
    print(type(file_upload))
    print(dir(file_upload))
#     print(file_upload)
# #     print(file_upload.value)
    

# #     preprocess_file(file_upload)
# #     create_layout()


file_upload = pn.widgets.FileInput()
file_upload.param.watch(set_subs, 'value')
pn.serve(pn.Column(file_upload, set_subs(file_upload)))

<class 'panel.widgets.input.FileInput'>
['_FileInput__params', '_Layoutable__abstract', '_Layoutable__params', '_Parameterized__db_print', '_Parameterized__params', '_Reactive__params', '_Renderable__abstract', '_Renderable__params', '_Syncable__abstract', '_Syncable__params', '_Viewable__params', '_Widget__abstract', '_Widget__params', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_location', '_add_parameter', '_align_param_value', '_apply_update', '_callbacks', '_change_coroutine', '_change_event', '_changing', '_cleanup', '_comm_change', '_comm_event', '_comms', '_css_classes_param_value', '_debounce', '_documents', '_dynamic_watchers'

<class 'param.parameterized.Event'>
['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_asdict', '_field_defaults', '_fields', '_make', '_replace', 'cls', 'count', 'index', 'name', 'new', 'obj', 'old', 'type', 'what']


In [64]:
file_upload.filename

'weights.txt'

In [ ]:
doc/
├── _static/
│   ├── embedded/
│   │   ├── deep_file
│   │   └── very/
│   │       └── deep/
│   │           └── folder/
│   │               └── very_deep_file
│   └── less_deep_file
├── about.rst
├── conf.py
└── index.rst

In [ ]:
f"""
        |___ sub-{sub['name']}
             |__ net
                 |__ sub-{sub['name']}_desc-{sub['desc']}_distances.tsv
                 |__ sub-{sub['name']}_desc-{sub['desc']}_distances.json
                 |__ sub-{sub['name']}_desc-{sub['desc']}_weights.tsv
                 |__ sub-{sub['name']}_desc-{sub['desc']}_weights.json
             |__ spatial
             |__ ts 
    
    """

In [72]:
def reset(event):
    file_input.disabled = False
    progress.active = False

file_input = pn.widgets.FileInput()
progress = pn.widgets.Progress(active=False)
file_input.jscallback(
    args={"progress": progress},
    value="""
        progress.active = true;
        source.disabled = true;
    """
)
file_input.param.watch(reset, "value")
col = pn.Column(progress, file_input)
col.servable()

Column(sizing_mode='stretch_width')
    [0] Progress(active=False)
    [1] FileInput(sizing_mode='stretch_width')

In [75]:
import io
import param
import panel as pn
import pandas as pd

from datetime import datetime, timedelta

import plotly.express as px

pn.extension('plotly')
pn.config.sizing_mode="stretch_width"

class VoltageApp(param.Parameterized):
    data = param.DataFrame()

    file_input = param.Parameter()

    def __init__(self, **params):
        self.param.file_input.default = pn.widgets.FileInput()
        super().__init__(**params)
        self.plotly_pane = pn.pane.Plotly(height=400, sizing_mode="stretch_width")

    @pn.depends("file_input.value", watch=True)
    def _parse_file_input(self):
        value = self.file_input.value
        if value:
            string_io = io.StringIO(value.decode("utf8"))
            self.data = pd.read_csv(string_io, parse_dates=["Time"])
        else:
            print("Error: FileInput.value is None!")

    @pn.depends('data', watch=True)
    def get_plot(self):
        df = self.data
        if df is None:
            return
        assert ("Voltage" in df.columns) and ("Time" in df.columns), "no columns voltage and time"
        df = (df.loc[df['Voltage'] != 'Invalid/Calib']).copy(deep=True)
        df['Voltage'] = df['Voltage'].astype(float)
        if "FubId" in df.columns:
            p = px.scatter(df, x="Time", y="Voltage", color="FubId")
        else:
            p = px.scatter(df, x="Time", y="Voltage")
        self.plotly_pane.object = p

    def view(self):
        return pn.Column(
            self.file_input,
            self.plotly_pane,
        )

voltage_app = VoltageApp()
voltage_app_view = voltage_app.view()


# Removing this line makes the app work
# voltage_app = VoltageApp()

pn.serve(pn.template.FastListTemplate(site="Panel", title="Download and Upload CSV File", main=[voltage_app_view]))

Launching server at http://localhost:58145


Callback failed for object named "" changing properties {'filename': 'weights.txt',
 'mime_type': 'text/plain',
 'value': b' 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.0'
          b'0000000e+00 1.33333333e-04 9.82666667e-03 6.13333333e-04 1.60000'
          b'000e-04 2.53333333e-04 1.06666667e-04 0.00000000e+00 0.00000000e'
          b'+00 3.70133333e-02 0.00000000e+00 0.00000000e+00 1.54666667e-03 '
          b'1.60000000e-04 1.88000000e-03 0.00000000e+00 5.06666667e-04 0.00'
          b'000000e+00 8.00000000e-04 5.20000000e-04 0.00000000e+00 5.733333'
          b'33e-04 9.33333333e-05 1.01333333e-03 8.26133333e-02 2.42000000e-'
          b'02 0.00000000e+00 9.33333333e-05 0.00000000e+00 7.60000000e-04 0'
          b'.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.000'
          b'00000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.0000000'
          b'0e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+0'
          b'0 0.00000000e+00 0

In [ ]:
class 

In [83]:
class F(param.Parameterized):
    f = param.FileSelector(path='../data/txt_files/*')
#     fs = param.MultiFileSelector(path='/usr/share/*')
    
f = F()
f.param.f.objects[:]

['../data/txt_files\\weights.txt']

In [104]:
class UploadFile(param.Parameterized):
    file_input = param.Parameter()
    
    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.FileInput(), **params)
        self.static_text = pn.widgets.StaticText()
    
    
    @pn.depends('file_input.value', watch=True)
    def _parse_file_input(self):
        value = self.file_input.value
        fname = self.file_input.filename
        
        print(value, fname)
    
    def view(self):
        return pn.Column(
            "## Upload and Plot Data",
            self.file_input,
            self.static_text)


app = UploadFile()
app.view()

pn.serve(app)

Launching server at http://localhost:55225


In [106]:
class VoltageApp(param.Parameterized):
#     data = param.DataFrame()
    
    file_input = param.Parameter()
    
    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.FileInput(), **params)
        self.static_text = pn.widgets.StaticText()
        
    @pn.depends("file_input.value", watch=True)
    def _parse_file_input(self):
        value = self.file_input.value
        fname = self.file_input.filename
        
        if value:
            check_file(fname, value)
        else:
            print("error")

#     @pn.depends('data', watch=True)
#     def get_plot(self):
#         df = self.data
#         if df is None:
#             return
#         assert ("Voltage" in df.columns) and ("Time" in df.columns), "no columns voltage and time"
#         df = (df.loc[df['Voltage'] != 'Invalid/Calib']).copy(deep=True)
#         df['Voltage'] = df['Voltage'].astype(float)
#         if "FubId" in df.columns:
#             p = px.scatter(df, x="Time", y="Voltage", color="FubId")
#         else:
#             p = px.scatter(df, x="Time", y="Voltage")
#         self.plotly_pane.object = p
        
    def view(self):
        return pn.Column(
            "## Upload and Plot Data",
            self.file_input,
#             self.plotly_pane,
        )
    
voltage_app = VoltageApp()

voltage_app_view = voltage_app.view()
pn.serve(voltage_app_view)

Launching server at http://localhost:65000


weights.txt b' 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.33333333e-04 9.82666667e-03 6.13333333e-04 1.60000000e-04 2.53333333e-04 1.06666667e-04 0.00000000e+00 0.00000000e+00 3.70133333e-02 0.00000000e+00 0.00000000e+00 1.54666667e-03 1.60000000e-04 1.88000000e-03 0.00000000e+00 5.06666667e-04 0.00000000e+00 8.00000000e-04 5.20000000e-04 0.00000000e+00 5.73333333e-04 9.33333333e-05 1.01333333e-03 8.26133333e-02 2.42000000e-02 0.00000000e+00 9.33333333e-05 0.00000000e+00 7.60000000e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000

In [99]:
from panel.viewable import Viewer

class EditableRange(Viewer):
    
    value = param.Range(doc="A numeric range.")
    
    width = param.Integer(default=300)
    
    def __init__(self, **params):
        self._start_input = pn.widgets.FloatInput()
        self._end_input = pn.widgets.FloatInput(align='end')
        super().__init__(**params)
        self._layout = pn.Row(self._start_input, self._end_input)
        self._sync_widgets()
    
    def __panel__(self):
        return self._layout
    
    @param.depends('value', 'width', watch=True)
    def _sync_widgets(self):
        self._start_input.name = self.name
        self._start_input.value = self.value[0]
        self._end_input.value = self.value[1]
        self._start_input.width = self.width//2
        self._end_input.width = self.width//2
        
    @param.depends('_start_input.value', '_end_input.value', watch=True)
    def _sync_params(self):
        self.value = (self._start_input.value, self._end_input.value)
    
range_widget = EditableRange(name='Range', value=(0, 10))

pn.Column(
    '## This is a custom widget',
    range_widget
)

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] FloatInput(name='Range', sizing_mode='stretch_width', width=150)
        [1] FloatInput(align='end', sizing_mode='stretch_width', value=10, width=150)